# Library/Module

In [ ]:
import math, time, random
import numpy as np    
from scipy import signal as sig    # Signal Processing
from scipy import interpolate    # Interpolation
import matplotlib.pyplot as plt    # Plot Tool
import matplotlib.mlab as mlab    # PSD Estimation

import cupy as cp    # For GPU
import chainer    # Neural Network Framework
import chainer.links as L
import chainer.functions as F
from chainer.dataset import convert
from chainer.datasets import TupleDataset
from chainer import serializers

from function.noise import *    # Noise Production
from function.Ringdown_signal import *    # Ringdown Signal Production
from function.matched_filter import *    # Matched Filtering

# Noise

## Source Code

You can produce pseudo noise data by running 'noise.py' .  
The source code is like below.

In [ ]:
def data_format(data_name):
    with open(data_name, 'r') as f:
        data = f.read()

    data = data.split('\n')
    del data[-1]

    for i in range(len(data)):
        data[i] = float(data[i])

    data = np.array(data)

    return data


def rand_sample(data, fs, L):
    # PSD Estimation
    dt = 1.0 / fs
    NFFT = fs
    
    psd, freqs = mlab.psd(data, Fs=fs, NFFT=NFFT)
    
    # Whitening
    Nt = L
    data2 = data
    
    hf = np.fft.rfft(data2)
    white_hf = hf / np.sqrt(psd / dt / 2)
    white_hf = np.fft.irfft(white_hf, Nt)
    
    # Randomization
    rand_hf = np.random.permutation(white_hf)
    rand_hf = np.fft.rfft(rand_hf)
    rand_hf = rand_hf * np.sqrt(psd / dt / 2)
    rand_hf = np.fft.irfft(rand_hf)
    
    norm_factor = np.median(np.abs(rand_hf)) / np.median(np.abs(data))
    rand_hf = rand_hf / norm_factor
    
    return rand_hf

data_format : Formating LIGO's raw data(.txt).  
rand_sample : Permutation test function for formated data.

## How to use

You have to set sampling frequency(fs) and data length(L).  
Here, sampling frequency of LIGO's raw data is 4096Hz, so fs = 4096.
And we use 1 second data for this time, so L = 4096.

In [ ]:
# Path of data
DATA_PATH = '../data/GW150914_4096Hz_1s.txt'

# Parameters of data
fs = 4096
L = 4096

# Loading and formating
raw_data = data_format(DATA_PATH)

# Permutation test
noise = noise(raw_data, fs=fs, L=L)

Then, you can get noise data(t-domain), and the size is (4096, ).

# Ringdown Signal

## Source Code

You can get Ringdown Signal by running 'Ringdown_signal.py' .  
The source code is like below.

In [ ]:
def Ringdown_signal_generator(m1, m2, a, t, A22, theta, tau_index):
    q = m2 / m1
    if q < 1:
        q = 1 / q
        
    nu = q / ((1 + q)**2)                         # Mass ratio
    
    m = round((m1 + m2) * 0.96, 1)     # Mass after the merger：4% of this mass is convert to GWs, so after mass is 96%.
    a = round(a, 3)
    
    t[:tau_index] = 0
    t[tau_index:] = t[tau_index:] - t[tau_index]      # Reshape of t

    # QNM parameter
    f, q = QNM_parameter(a, m)
    
    f_22 = f[0]
    f_33 = f[1]
    f_44 = f[2]
    q_22 = q[0]
    q_33 = q[1]
    q_44 = q[2]
    
    # Amplitude of spherical harmonics
    Y22 = plus_GW_spherical_harmonics(theta, 2, 2)
    Y33 = plus_GW_spherical_harmonics(theta, 3, 3) / Y22
    Y44 = plus_GW_spherical_harmonics(theta, 4, 4) / Y22
    
    # Amplitude
    A33 = Y33 * A22 * 0.44 * (1- 4*nu)**0.45
    A44 = Y44 * A22 *(5.4 * (nu - 0.22)**2 + 0.04)
    
    # Signal
    
    signal_22 = A22 * np.exp(-math.pi * f_22 / q_22 * t) * np.sin(2 * math.pi * f_22 * t)
    signal_33 = A33 * np.exp(-math.pi * f_33 / q_33 * t) * np.sin(2 * math.pi * f_33 * t)
    signal_44 = A44 * np.exp(-math.pi * f_44 / q_44 * t) * np.sin(2 * math.pi * f_44 * t)
    signal = signal_22 + signal_33 + signal_44
    
    return np.real(signal)
    
def QNM_parameter(a,m):
    f_sig = np.array([[1.5251, -1.1568, 0.1292], [1.8956, -1.3043, 0.1818], [2.3000, -1.5056, 0.2244]])     # 22, 33, 44 mode、row:mode, columun:index
    q_sig = np.array([[0.7000, 1.4187, -0.4990], [0.9000, 2.3430, -0.4810], [1.1929, 3.1191, -0.4825]])
    
    f = np.zeros(3)
    q = np.zeros(3)
    
    for i in range(len(f_sig)):
        f[i] = 32 * (f_sig[i, 0] + f_sig[i, 1] * (1 - a)**f_sig[i, 2]) / m * 10**3
        q[i] = q_sig[i, 0] + q_sig[i, 1] * (1 - a)**q_sig[i, 2]
        
    return f, q

def plus_GW_spherical_harmonics(theta, l, m):       # theta:Inclination, l,m:QNM mode
    s = -2
    phi = 0
    
    y1 = spinweightedsphericalharmonics(theta, phi, l, m, s)
    y2 = (-1)**(-1 * s + m) * np.conj(spinweightedsphericalharmonics(theta, phi, l, m, s))
    
    Ylm = y1 + (-1)**l * y2
    
    return Ylm

def spinweightedsphericalharmonics(theta, phi, l, m, s):
    n = -1 * s
    d = small_dmatrix(theta, l, m, n)
    
    sYlm = (-1)**s * np.sqrt((2*l + 1) / 4 * math.pi) * d * np.exp(1j * m * phi)
    
    return sYlm

def small_dmatrix(theta, l, m, n):
    maxk = min(l - m, l - n) + 1
    SUM = 0
    
    for k in range(maxk):
        
        SUM += (-1)**k * (np.sin(theta / 2))**(2*l - m - n - 2*k) * (np.cos(theta / 2))**(m + n + 2*k) / (math.factorial(k) * math.factorial(l - m - k) * math.factorial(l - n - k) * math.factorial(m + n + k))
        
    d = (-1)**(l - n) * np.sqrt(math.factorial(l + m) * math.factorial(l - m) * math.factorial(l + n) * math.factorial(l - n)) * SUM
    
    return d

Ringdown_signal_generator : Main function.  
QNM_parameter : Calculate QNM parameters(frequency and Q-factor).  
plus_GW_spherical_harmonics : Calculate spherical harmonics of GW.  
spinweightedsphericalharmonics : Calculate spin-weighted-spherical-harmonics.  
small_dmatrix : Calculate Wigner's small-D-matrix.

## How to use

You have to set 6 parameters(each mass of two BHs, spin, amplitude, inclination, arrival time of GW) in advance.

In [ ]:
# Parameters
# GW parameter
m1 = 29
m2 = 36
a = 0.68
theta = 3 * math.pi /2
A22 = 6.0 * 10**(-23)
tau_index = 2048    # Index of arrival time

fs = 4096
L = 4096
dt = 1.0 / L

t = np.linspace(0, 1.0 - dt, L)

# Signal
signal = Ringdown_signal_generator(m1, m2, a, t, A22, theta, tau_index)

Then, you can get ringdown signal(t-domain), and the size is (4096, ).

# Matched Filtering

## Source Code

You can calculate matched-filter $m$ by running 'matched_filter.py'.  
The source code is like below.  
※ This code is simpler version($\nu$ is a scalar).

In [ ]:
def matched_filter(noise, signal, fs, massvector, kerrvector, tauvector, nu, thetavector):
    
    # Preparation
    dt = 1.0 / fs
    L = len(noise)
    t = np.linspace(0, 1, fs)
    NFFT = fs         # Length of fourier transoformation
    
    # PSD estimation
    psd, freqs = mlab.psd(noise, Fs=fs, NFFT=NFFT)
    df = 1.0           # bin
    Lf = len(freqs)
    
    # Signal + Noise
    s = noise + signal
    w = sp.signal.hann(L)           # Window function(hunn)
    w = w / np.mean(w)
    s_fft = np.fft.rfft(s * w)/L
    
    ## Matched Filter
    massvector = np.round(massvector, 3)
    kerrvector = np.round(kerrvector, 3)
    tauvector = np.round(tauvector, 3)
    
    ml = len(massvector)
    kl = len(kerrvector)
    tl = len(tauvector)
    thl = len(thetavector)
    
    # Templates
    templates = np.zeros((L, ml, kl, thl))

    for i in range(ml):
        for j in range(kl):
            for k in range(thl):                   
                #--------QNM parameter--------#
                f, q = QNM_parameter(kerrvector[j], massvector[i])

                f_22, f_33, f_44 = f[0], f[1], f[2]
                q_22, q_33, q_44 = q[0], q[1], q[2]
                    
                #--------Amplitude of spherical harmonics--------#
                Y22 = plus_GW_spherical_harmonics(thetavector[k], 2, 2)
                Y33 = plus_GW_spherical_harmonics(thetavector[k], 3, 3) / Y22
                Y44 = plus_GW_spherical_harmonics(thetavector[k], 4, 4) / Y22

                #--------Amplitude--------#
                A22 = 1.0
                A33 = Y33 * A22 * 0.44 * (1- 4*nu)**0.45
                A44 = Y44 * A22 *(5.4 * (nu - 0.22)**2 + 0.04)
                    
                #--------Templates-------#

                temp_22 = A22 * np.exp(-math.pi * f_22 / q_22 * t) * np.sin(2 * math.pi * f_22 * t)
                temp_33 = A33 * np.exp(-math.pi * f_33 / q_33 * t) * np.sin(2 * math.pi * f_33 * t)
                temp_44 = A44 * np.exp(-math.pi * f_44 / q_44 * t) * np.sin(2 * math.pi * f_44 * t)
                    
                templates[:, i, j, k] = temp_22 + temp_33 + temp_44
                
                
    # Matched Filter
    #--------Normalization-------#
    temp_fft = np.zeros((int(NFFT/2.0+1), ml, kl, thl))
    sigma_sqr = np.zeros((ml, kl, thl))
    templates_cal = np.zeros((L, ml, kl, thl))
    temp_fft_cal = np.zeros((int(NFFT/2.0+1), ml, kl, thl))
    sigma_sqr_cal = np.zeros((ml, kl, thl))

    w = sp.signal.hann(NFFT)
    w = w / np.mean(w)
    
    for i in range(ml):
        for j in range(kl):
            for k in range(thl):
                temp_fft[:, i, j, k] = np.fft.rfft(templates[:, i, j, k] * w) / NFFT
                sigma_sqr[i, j, k] = 4 * np.real(np.sum((temp_fft[1:, i, j, k] * np.conj(temp_fft[1:, i, j, k]) / psd[1:]) * df))
                templates_cal[:, i, j, k] = templates[:, i, j, k] / np.sqrt(sigma_sqr[i, j, k])
                temp_fft_cal[:, i, j, k] = np.fft.rfft(templates_cal[:, i, j, k] * w) / NFFT
                sigma_sqr_cal[i,j,k] = 4 * np.real(np.sum((temp_fft_cal[1:, i, j, k] * np.conj(temp_fft_cal[1:, i, j, k]) / psd[1:]) * df))
    
    matchedfilter_cal = np.zeros((ml, kl, tl, thl))
                
    for i in range(ml):
        for j in range(kl):
            for k in range(tl):
                for l in range(thl):
                    matchedfilter_cal[i, j, k, l] = 4 * np.real(np.sum((s_fft[1:] * np.conj(temp_fft_cal[1:, i, j, l]) * np.exp(2 * math.pi *1j * freqs[1:] * tauvector[k]) / psd[1:]) * df))
                    
    return np.abs(matchedfilter_cal)


## How to use

You have to set 4 search parameters here(In fact, 5 parameters including $\nu$).

In [ ]:
# Search parameters
massvector = np.linspace(10,100,10)
kerrvector = np.linspace(0.48,0.88,5)
tauvector = np.linspace(0.3,0.7,5)
thetavector = np.array([0, math.pi/4, math.pi/2, math.pi*5/4, math.pi*3/2])

q = m2/m1
nu = q/((1+q)**2)    # Mass ratio is fixed here.

# Matched Filtering
SNR = matched_filter(noise, signal, fs, massvector, kerrvector, tauvector, nu, thetavector)

Then, you can get matched-filter $m$, so you go on to statistical testing.

## Example of statistical testing(DetRate vs FAR)

In [ ]:
kmax = 10000    # number of loop

# Preparation
SNR_storage = np.zeros(kmax)
SNR_storage_nosignal = np.zeros(kmax)

#--------Preparation for noise--------#
data_name = 'GW150914_4096Hz_1s.txt'

# Parameters
L = 4096
fs = 4096
dt = 1.0 / fs

t = np.linspace(0, 1 - dt, L)

strain = data_format(data_name)


#--------Preparation for ringdown signal--------#
# Parameters
a = 0.68       # Spin
theta = 3 * math.pi / 2      # Inclination
A22 = 1.000000000000001e-22      # Amplitude of fundamental mode
tau_index = 2048

m1 = 29
m2 = 36

# Ringdown signal
signal = Ringdown_signal_generator(m1, m2, a, t, A22, theta, tau_index)

#--------Matched Filtering--------#

start_time = time.time()

# Search parameters
massvector = np.linspace(10,100,10)
kerrvector = np.linspace(0.48,0.88,5)
tauvector = np.linspace(0.3,0.7,5)
thetavector = np.array([0, math.pi/4, math.pi/2, math.pi*5/4, math.pi*3/2])

q = m2/m1
nu = q/((1+q)**2)    # Mass ratio is fixed here


for i in range(kmax):
    noise = rand_sample(strain, fs, L)
    
    # Including ringdown signal
    SNR = matched_filter(noise, signal, fs, massvector, kerrvector, tauvector, nu, thetavector)
    
    # Not including ringdown signal
    no_signal = np.zeros(4096)
    SNR_nosignal = matched_filter(noise, no_signal, fs, massvector, kerrvector, tauvector, nu, thetavector)
    
    # Max SNR
    SNR_storage[i] = np.amax(SNR)
    SNR_storage_nosignal[i] = np.amax(SNR_nosignal)
    
#--------Plotting-------#
# Setting of bias
ref_min = np.amin(SNR_storage_nosignal)
ref_max = np.amax(SNR_storage)
ref_val = np.linspace(ref_min, ref_max, 10000)

# Preparation for testing
num = np.zeros((len(ref_val), 2))

# Testing
for i in range(len(ref_val)):
    num[i,0] = np.sum(ref_val[i] < SNR_storage)
    num[i,1] = np.sum(ref_val[i] < SNR_storage_nosignal)
    
# Number to Probability
num = num / kmax

# Plot
plt.figure(figsize=(5, 5), dpi=100)
plt.scatter(num[:,1], num[:,0], marker='.', s=20)
plt.title('DetRate vs FAR')
plt.xlabel('False Alarm Rate')
plt.ylabel('Detection Rate')
plt.grid(True)
plt.show()

Then, you can get results of Matched Filtering.

# Convolutional Neural Network

## Train / Test Data

You can make train/test dataset by running 'make_train_test.py'.  
The source code is like below.

In [ ]:
data_path = 'data/GW150914_4096Hz_1s.txt'

# Preparation

L = 4096
fs = 4096
dt = 1.0 / fs

t = np.linspace(0, 1 - dt, L)

# Strain formating

strain = data_format(data_path)


# GW parameters

a = 0.68       # Spin
theta = 3 * math.pi / 2      # Inclination
A22 = 6.000000000000001e-23      # Amplitude of the fundamental(22) mode

train_m1 = np.linspace(5, 100, 20)
train_m2 = train_m1
test_m1 = np.linspace(7.5, 97.5, 19)
test_m2 = test_m1

tau_index = np.array([1229, 1639, 2048, 2458, 2867])     # Index of arrival time

# Preparation for datasets

train_data = np.zeros((2100, 4101))
test_data = np.zeros((1900, 4101))

# Train Data

loop_train = 0

for i1 in range(len(train_m1)):
    for i2 in range(len(train_m2)):
        for i3 in range(len(tau_index)):
            if train_m1[i1] <= train_m2[i2]:
                ringdown = Ringdown_signal_generator(train_m1[i1], train_m2[i2], a, t, A22, theta, tau_index[i3])
                strain = rand_sample(strain, fs, L)
                signal = ringdown + strain

                # Including ringdown
                
                train_data[loop_train, 0:L] = signal
                train_data[loop_train, L] = train_m1[i1]
                train_data[loop_train, L+1] = train_m2[i2]
                train_data[loop_train, L+2] = tau_index[i3]
                train_data[loop_train, L+3] = 1
                train_data[loop_train, L+4] = 0

                loop_train += 1

                # Not including ringdown

                train_data[loop_train, 0:L] = strain
                train_data[loop_train, L] = train_m1[i1]
                train_data[loop_train, L+1] = train_m2[i2]
                train_data[loop_train, L+2] = tau_index[i3]
                train_data[loop_train, L+3] = 0
                train_data[loop_train, L+4] = 1

                loop_train += 1

                print('i1:' + str(i1) + ', i2:' + str(i2) + ', i3:' + str(i3) + ', Train:' + str(loop_train / 46560 * 100) + '%')

# Test Data

loop_test = 0

for i1 in range(len(test_m1)):
    for i2 in range(len(test_m2)):
        for i3 in range(len(tau_index)):
            if test_m1[i1] <= test_m2[i2]:
                ringdown = Ringdown_signal_generator(test_m1[i1], test_m2[i2], a, t, A22, theta, tau_index[i3])
                strain = rand_sample(strain, fs, L)
                signal = ringdown + strain

                # Including ringdown
                
                test_data[loop_test, 0:L] = signal
                test_data[loop_test, L] = test_m1[i1]
                test_data[loop_test, L+1] = test_m2[i2]
                test_data[loop_test, L+2] = tau_index[i3]
                test_data[loop_test, L+3] = 1
                test_data[loop_test, L+4] = 0

                loop_test += 1

                # Not including ringdown

                test_data[loop_test, 0:L] = strain
                test_data[loop_test, L] = test_m1[i1]
                test_data[loop_test, L+1] = test_m2[i2]
                test_data[loop_test, L+2] = tau_index[i3]
                test_data[loop_test, L+3] = 0
                test_data[loop_test, L+4] = 1

                loop_test += 1

                print('i1:' + str(i1) + ', i2:' + str(i2) + ', i3:' + str(i3) + ', Test:' + str(loop_test / 1900 * 100) + '%')

                
# Saving train/test datasets(Numpy array)

np.savetxt('data/train_data.csv', train_data, delimiter=',')
np.savetxt('data/test_data.csv', test_data, delimiter=',')

You can set GW parameters which you want.

## Modeling

This is the CNN model I used on my master thesis.  
You can change hyperparameters(ksize:kernel size, stride, ...).

In [ ]:
# CNN Block

class RingdownBlock(chainer.Chain):
    def __init__(self, in_channels, out_channels, ksize, stride):
        super(RingdownBlock, self).__init__()
        with self.init_scope():
            self.conv = L.ConvolutionND(ndim=1, in_channels=in_channels, out_channels=out_channels, ksize=ksize, stride=stride)
            self.bn = L.BatchNormalization(out_channels)
            
    def __call__(self, x):
        h = self.conv(x)
        h = self.bn(h)
        
        return h
      
# CNN modeling

class RingdownNet1(chainer.Chain):
    def __init__(self, class_labels=2):
        super(RingdownNet1, self).__init__()
        with self.init_scope():
            self.conv1 = RingdownBlock(1, 16, 9, 1)
            self.conv2 = RingdownBlock(16, 32, 7, 1)
            self.conv3 = RingdownBlock(32, 64, 7, 1)
            self.affine1 = L.Linear(None, 32)
            self.affine2 = L.Linear(None, class_labels)
            #self.bn1 = L.BatchNormalization(32)
          
            
    def __call__(self, x):
        # 1st layer(CNN)
        h = self.conv1(x)
        h = F.max_pooling_nd(h, ksize=4, stride=4)
        h = F.relu(h)
        
        # 2nd layer(CNN)
        h = self.conv2(h)
        h = F.max_pooling_nd(h, ksize=4, stride=4)
        h = F.relu(h)
        
        # 3rd layer(CNN)
        h = self.conv3(h)
        h = F.max_pooling_nd(h, ksize=4, stride=4)
        h = F.relu(h)
        
        # 4th layer(fully-connected)
        h = self.affine1(h)
        #h = self.bn1(h)
        h = F.relu(h)
        
        # Output layer
        h = self.affine2(h)
        
        return h

## Training(on Google Colaboratory)

The process described below is executed on Google Colaboratory.  
At first, I define a function about 'random seed'.  
This function helps you manage 'random seed' in order not to change on training.

In [ ]:
def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)    # For numpy
    cp.random.seed(seed)    # For cupy
    
reset_seed(0)

### Installation and Setup of Chainer

In [ ]:
!curl https://colab.chainer.org/install | sh -

Now you can use chainer on Google Colaboratory.  
So, check it.

In [ ]:
GPU = True
GPU_ID = 0

print('cuda.available:', chainer.cuda.available)
print('cuda.cudnn_enabled:',chainer.cuda.cudnn_enabled)

### Loading train/test datasets

At first, you have to mount specified directory.

In [ ]:
# Mounting specified directory on Google Drive

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Then, URL for activation will be displayed twice.  
You have to click the URL, login Google and copy activation key.  
Finally, fill the activation key in the window on Google Colaboratory.

In [ ]:
# Drive mean root directory of  google drive

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/NAME_OF_DIRECTORY/

Then, you've completed activation.  
So, go on to the next step(Loading datasets).

In [ ]:
# Loading train/test datasets

PATH_TRAIN = 'drive/DeepGW/train_data.csv'
PATH_TEST = 'drive/DeepGW/test_data.csv'

train = np.loadtxt(PATH_TRAIN, delimiter=',')
test = np.loadtxt(PATH_TEST, delimiter=',')

# Reshape

x_train, _, t_train, _ = np.split(train, [4096, 4099, 4100], axis=1)
x_train = x_train.reshape(len(x_train), 1, 4096)
x_test, _, t_test, _ = np.split(test, [4096, 4099, 4100], axis=1)
x_test = x_test.reshape(len(x_test), 1, 4096)


# Reshape for mini batch

t_train = t_train.reshape(len(t_train))
t_test = t_test.reshape(len(t_test))

# Numpy -> Cupy

x_train, t_train, x_test, t_test = cp.asarray(x_train), cp.asarray(t_train), cp.asarray(x_test), cp.asarray(t_test)

# Mini batch

train_data = TupleDataset(x_train, t_train)    # Datasets -> Tuple
test_data = TupleDataset(x_test, t_test)

MINIBATCH_SIZE = 128
train_iter = chainer.iterators.SerialIterator(train_data, MINIBATCH_SIZE)    # Iterator
test_iter = chainer.iterators.SerialIterator(test_data, MINIBATCH_SIZE)

Now, you've gotten train/test datasets for cupy/cuda.

### Training/Testing

In [ ]:
# Instance
reset_seed(0)
model = RingdownNet1()

if GPU:
  chainer.cuda.get_device(GPU_ID).use()
  model.to_gpu(GPU_ID)

# Optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

You can change a kind of optimizers and hyper parameters(learning rate and so on...).  
Now, we are ready to train the model.  
Run next code or 'Trainer' , and training is executed.

In [ ]:
#  Train and Test

from chainer.cuda import to_cpu

EPOCH = 10    # Setting training epoch

train_loss_list = cp.zeros(EPOCH)
train_accuracy_list = cp.zeros(EPOCH)

while train_iter.epoch < EPOCH:
    batch = train_iter.next()
    x_array, t_array = convert.concat_examples(batch, GPU_ID)
    x = chainer.Variable(x_array.astype(cp.float32))
    t = chainer.Variable(t_array.astype(cp.int32))

    y = model(x)
    loss_train = F.softmax_cross_entropy(y, t)
    model.cleargrads()
    loss_train.backward()
    optimizer.update()
    
    # Tesiting generalization ability by epoch
    if train_iter.is_new_epoch:  # When 1 epoch is finished

        # Loss
        print('epoch:{:02d} train_loss:{:.06f} '.format(
            train_iter.epoch, float(to_cpu(loss_train.data))), end='')
        train_loss_list[train_iter.epoch-1] = loss_train.data

        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            x_test_array, t_test_array = convert.concat_examples(test_batch, GPU_ID)
            x_test = chainer.Variable(x_test_array.astype(cp.float32))
            t_test = chainer.Variable(t_test_array.astype(cp.int32))

            # Forward test datasets
            y_test = model(x_test)

            # Calculate loss
            loss_test = F.softmax_cross_entropy(y_test, t_test)
            test_losses.append(to_cpu(loss_test.array))

            # Calculate accuracy
            accuracy = F.accuracy(y_test, t_test)
            accuracy.to_cpu()
            test_accuracies.append(accuracy.array)

            if test_iter.is_new_epoch:
                test_iter.reset()
                break

        print('test_loss:{:.04f} test_accuracy:{:.06f}'.format(
            np.mean(test_losses), np.mean(test_accuracies)))

    
print('Training is finished')

# Visualization of  training process
epo = np.linspace(1, EPOCH, EPOCH)
plt.plot(epo, cp.asnumpy(train_loss_list))
plt.show()

When you want to save parameters(weight and bias), run next code.  
Then, you've gotten '.npz' file.

In [ ]:
# Saving parameters

model.to_cpu()
SAVE_MODEL = '/content/drive/NAME_OF_DIRECTORY/NPZ_FILE_NAME.npz'
serializers.save_npz(SAVE_MODEL, model)
print('Saved')

### Loading saved parameters

In [ ]:
# Loading savd parameters

SAVE_MODEL = '/content/drive/NAME_OF_DIRECTORY/NPZ_FILE_NAME.npz'
model = RingdownNet1()
serializers.load_npz(SAVE_MODEL, model, strict=False)